# Tokenize and save data

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.text import *

In [3]:
import pandas as pd
from fastai_data import *

In [4]:
from encode_data import *

In [5]:
from midi_data import *

## Preparing the data

In [6]:
source_dir = 'midi_transcribe_v1_simple'
path = Path(f'data/midi/{source_dir}/')
csv_path = path/f'{source_dir}.csv'
path.ls()[:5]

[PosixPath('data/midi/midi_transcribe_v1_simple/ecomp'),
 PosixPath('data/midi/midi_transcribe_v1_simple/classic_piano'),
 PosixPath('data/midi/midi_transcribe_v1_simple/cprato'),
 PosixPath('data/midi/midi_transcribe_v1_simple/tmp_pop'),
 PosixPath('data/midi/midi_transcribe_v1_simple/tmp')]

In [7]:
csv = pd.read_csv(csv_path)
csv = csv.loc[csv[source_dir].notna()];

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (5,13,14,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
def get_df_files(csv):
    files = csv[source_dir]
    flist = [Path(f) for f in files.values]
    flist = [f for f in flist if f.exists()]
    return flist

### Load Pop Dataset

In [8]:
pop_csv = csv.loc[csv.source.isin(['midiworld', 'hooktheory', 'freemidi', 'cprato'])]
pop_files = get_df_files(pop_csv)

In [19]:
data = TextLMDataBunch.load(path, cache_name='tmp_pop', bs=4, bptt=500)

In [20]:
vocab = data.vocab

Play csv

In [52]:
fp = pop_files[500]
with open(fp, 'r') as f: 
    ftext = f.read()
len(str2seq(ftext)), fp

(PosixPath('data/midi/midi_transcribe_v1_simple/hooktheory/pianoroll/y/yasuaki-fujita/mega-man-3---ending/chorus_key.txt'),
 257)

In [42]:
note_stream = str2stream(ftext);
note_stream.show('midi')

In [51]:
enc_dec_text = vocab.textify(vocab.numericalize(ftext.split(' ')))
note_stream = str2stream(enc_dec_text)
note_stream.show('midi')

Play databunch

In [21]:
ob = data.one_batch()

In [31]:
idx = 0
note_stream = str2stream(vocab.textify(ob[1][0]))
note_stream.show('midi')

## Create Classical Dataset

In [9]:
# csv_filter = csv.loc[csv[f'{source_dir}_timesteps'] < 20000]
clc_csv = csv.loc[csv.source.isin(['classical_piano', 'ecomp'])]
clc_files = get_df_files(clc_csv)

In [10]:
data = TextLMDataBunch.load(path, cache_name='tmp_clc', bs=4, bptt=500)

In [11]:
vocab = data.vocab

Play csv

In [12]:
fp = clc_files[500]
with open(fp, 'r') as f: 
    ftext = f.read()
len(str2seq(ftext)), fp

(1080,
 PosixPath('data/midi/midi_transcribe_v1_simple/ecomp/2009/Staupe01.txt'))

In [13]:
note_stream = str2stream(ftext);
note_stream.show('midi')

In [61]:
enc_dec_text = vocab.textify(vocab.numericalize(ftext.split(' ')))
note_stream = str2stream(enc_dec_text)
note_stream.show('midi')

### Locate transposed file and play

In [14]:
clc_csv[clc_csv[source_dir] == str(fp)].midi_transform_v1.values

array(['data/midi/midi_transform_v1/ecomp/2009/Staupe01.MID'], dtype=object)

In [15]:
play_midi('data/midi/midi_transform_v1/ecomp/2009/Staupe01.MID')

Play databunch

In [62]:
ob = data.one_batch()

In [63]:
idx = 0
note_stream = str2stream(vocab.textify(ob[1][0]))
note_stream.show('midi')

't'
